# Projeto de Machine Learning: Diagnóstico de Câncer de Mama

## Introdução
Este projeto tem como objetivo desenvolver modelos preditivos capazes de classificar diagnósticos de câncer de mama como **malignos** ou **benignos**, utilizando o clássico conjunto de dados **Breast Cancer Wisconsin (Diagnostic)**, disponível no Kaggle.

O foco principal é minimizar a quantidade de **falsos negativos** — ou seja, evitar que casos malignos sejam classificados incorretamente como benignos. Esse cuidado é especialmente importante em contextos clínicos, onde a falha na detecção de um tumor pode ter consequências graves.

### Etapas do projeto:
- Análise exploratória e tratamento dos dados;
- Seleção de variáveis utilizando diferentes técnicas:
  - Correlação,
  - PCA (Principal Component Analysis),
  - RFE (Recursive Feature Elimination),
  - SelectKBest;
- Treinamento de diversos algoritmos de classificação:
  - Regressão Logística,
  - Random Forest,
  - XGBoost,
  - Support Vector Classifier (SVC),
  - K-Nearest Neighbors (KNN);
- Avaliação e comparação dos modelos com base nas métricas:
  - **Recall**,
  - **F1-Score**,
  - **Precision**,
  - **Accuracy**,
  - **AUC-ROC**;

Ao final, os resultados obtidos são comparados e discutidos, com destaque para o modelo que melhor atende ao objetivo principal: **reduzir os falsos negativos sem comprometer o desempenho geral da classificação.**

## Importando as biliotecas

In [ ]:
import os
import sys
import joblib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import shap
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFECV, SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    RocCurveDisplay, accuracy_score, auc, classification_report,
    confusion_matrix, f1_score, precision_score, recall_score,
    roc_auc_score, roc_curve
)
from sklearn.model_selection import RepeatedStratifiedKFold, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.svm import SVC

sys.path.append(os.path.abspath(".."))
from src.feature_selection import feature_selection_rfe_xgb
from src.model_evaluation import (
    make_pipeline, evaluate_model, plot_confusion_matrix, train_and_evaluate_model
)
from src.models import (
    plot_multiple_roc_auc, train_knn, train_logistic_regression,
    train_random_forest, train_svc, train_xgboost
)

from src.utils import (
    load_data, plot_correlation_heatmap, plot_swarm_features,
    plot_violin_features, show_basic_info, split_data
)

from sklearn.neighbors import KNeighborsClassifier

import shap

plt.style.use('seaborn-v0_8-darkgrid')

## Carregando e explorando os dados

In [ ]:
df = load_data("../data/breast cancer kaggle.csv")

In [ ]:
df.head()

In [ ]:
show_basic_info(df)

# Pré-processamento dos dados

## Codificando dados categóricos

In [ ]:
df['diagnosis'] = df['diagnosis'].map({'M': 1, 'B': 0})

## Checando se algum ID se repete

In [ ]:
df['id'].duplicated().sum()

## Verificando valores nulos

In [ ]:
df.isnull().sum()

In [ ]:
y = df['diagnosis']

drop_list = [
    'Unnamed: 32',
    'id',
    'diagnosis'
]

X = df.drop(drop_list, axis = 1 )
X.head()

## Observando a distribuição das classes no dataset

In [ ]:
bars = sns.countplot(x=y, hue=y, palette=['#1f77b4', '#ff7f0e'], legend=False)
plt.title("Distribuição de Casos de Câncer de Mama", fontsize=14, pad=20)

for bar in bars.patches:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, height, '{:.0f}'.format(height),
             va='bottom', ha='center', fontsize=11)
    
plt.xlabel(y.name, fontsize=11)
plt.ylabel('Frequência', fontsize=11)
plt.legend(title="Classe", labels=["Benigno (0)", "Maligno (1)"], loc="upper right")
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.show()

In [ ]:
unique, counts = np.unique(y, return_counts=True)

total = len(y)
percentages = counts / total * 100

plt.figure(figsize=(6, 6))
plt.pie(counts, labels=[f'Classe {int(u)}' for u in unique], autopct='%1.2f%%', colors=['#1f77b4', '#ff7f0e'],
        wedgeprops={'edgecolor': 'black', 'linewidth': 1}, textprops={'fontsize': 14})

plt.show()

Observa-se que a distribuição das classes está desbalanceada, com uma predominância de casos benignos em relação aos malignos. Esse desbalanceamento será considerado durante a construção e avaliação dos modelos preditivos.

## Visualização

In [ ]:
# Primeiras quinze features
plot_violin_features(X, y, start=0, end=15)

In [ ]:
# Últimas quinze features
plot_violin_features(X, y, start=15, end=30)

In [ ]:
plot_swarm_features(X, y, start=0, end=10, size = 3) # Primeiras 10 features

In [ ]:
plot_swarm_features(X, y, start=10, end=20, size=2) # Próximas 10 features

In [ ]:
plot_swarm_features(X, y, start=20, end=31, size=3) # Últimas 10 features

In [ ]:
plot_correlation_heatmap(X, figsize=(18, 18))

## Dividindo o dataset em conjunto de treino e teste

In [ ]:
X_train, X_test, y_train, y_test = split_data(X, y)

## **Feature selection com correlação**

In [ ]:
drop_list = ['perimeter_mean','radius_mean','compactness_mean','concave points_mean','radius_se','perimeter_se',
             'radius_worst','perimeter_worst','compactness_worst', 'concave points_worst','compactness_se', 
             'concave points_se','texture_worst','area_worst', 'fractal_dimension_mean', 'concavity_worst', 'texture_se']

In [ ]:
X_train_fs = X_train.drop(columns=drop_list)
X_test_fs = X_test.drop(columns=drop_list)

Observamos agora que não há mais features altamente correlacionadas.

In [ ]:
plot_correlation_heatmap(X_train_fs, figsize=(10, 10))

# Treinamento e Avaliação

## **Regressão Logística**

In [ ]:
results_lr, model_lr = train_logistic_regression(
    X_train_fs, X_test_fs, y_train, y_test,
    model_name="Regressão Logística"
)

In [ ]:
print(results_lr.to_string(index=False))

## Cross Validation

In [ ]:
print("Validação Cruzada Regressão Logística:")
print(evaluate_model(
    LogisticRegression(class_weight='balanced'), 
    X_train_fs, 
    y_train
))

## **Random Forest**

In [ ]:
results_rf, model_rf = train_random_forest(
    X_train_fs, X_test_fs, y_train, y_test,
    model_name="Random Forest (FS Corr)"
)

In [ ]:
print(results_rf.to_string(index=False))

## Cross Validation

In [ ]:
print("Validação Cruzada Random Forest:")
print(evaluate_model(
    RandomForestClassifier(class_weight='balanced'), 
    X_train_fs, 
    y_train,
    scaling=False
))

## **Xgboost**

In [ ]:
results_xgb, model_xgb = train_xgboost(
    X_train_fs, X_test_fs, y_train, y_test,
    optimize_hyperparams=False
)

In [ ]:
results_xgb

## Cross Validation

In [ ]:
print("\nValidação Cruzada - XGBoost (FS):")
print(evaluate_model(
    XGBClassifier(objective='binary:logistic', eval_metric='aucpr'),
    X_train_fs, 
    y_train,
    scaling=False
))

## Encontrando novos parâmetos com Randomized Search

In [ ]:
results_xgb_opt, xgb_opt_model, xgb_search = train_xgboost(
   X_train_fs, X_test_fs, y_train, y_test,
    optimize_hyperparams=True
)

In [ ]:
results = pd.concat([results_xgb, results_xgb_opt], ignore_index=True)
print(results.to_string(index=False))

## Cross Validation

In [ ]:
print("\nValidação Cruzada - XGBoost (Hiperparâmetros otimizados)")
print(evaluate_model(
    xgb_opt_model,
    X_train_fs,
    y_train,
    scaling=False
))

## Avaliação ROC/AUC com Cross Validation

In [ ]:
params_base = {
    'objective': 'binary:logistic',
    'random_state': 42,
    'n_estimators': 100,
}

params_opt = xgb_search.best_params_

models = {
    'XGB Padrão': XGBClassifier(**params_base),
    'XGB Otimizado': XGBClassifier(**params_opt)
}

plot_multiple_roc_auc(models, X_train_fs, y_train.values)

## Análise de importância das variáveis com SHAP

In [ ]:
feature_names = X_train_fs.columns.tolist()

xgb_classifier = xgb_opt_model

explainer = shap.TreeExplainer(
    xgb_classifier, 
    feature_names=feature_names
)

shap_values = explainer.shap_values(X_test_fs)

shap.summary_plot(
    shap_values, 
    features=X_test_fs, 
    feature_names=feature_names, 
    plot_type='bar',
    show=False
)

plt.title("Importância de features - XGBoost otimizado", fontsize=14)
plt.tight_layout()
plt.show()

## **SVC**

In [ ]:
results_svc, model_svc = train_svc(
    X_train_fs, X_test_fs, y_train, y_test,
    model_name="SVC (FS Corr)"
)

In [ ]:
print(results_svc.to_string(index=False))

## Cross Validation

In [ ]:
print("Relatório de classificação - SVC:")
print("Validação Cruzada SVC:")
print(evaluate_model(
    SVC(class_weight='balanced'), 
    X_train_fs, 
    y_train
))

## **KNN**

In [ ]:
results_knn, model_knn = train_knn(
    X_train_fs, X_test_fs, y_train, y_test,
    model_name="KNN (FS Corr)"
)

In [ ]:
print(results_knn.to_string(index=False))

## Cross Validation

In [ ]:
print("Relatório de classificação - KNN:")
print("Validação Cruzada KNN:")
print(evaluate_model(
    KNeighborsClassifier(n_neighbors=3), 
    X_train_fs, 
    y_train
))

# Seleção de variáveis com Recursive Feature Elimination (RFE) usando XGBoost

In [ ]:
X_train, X_test, y_train, y_test = split_data(X, y)

xgb_fs = XGBClassifier(eval_metric='logloss', random_state=1)
xgb_clf = XGBClassifier(eval_metric='logloss', random_state=1)

rfe = RFECV(estimator=xgb_fs, scoring='recall', cv=5, n_jobs=-1)
pipeline = Pipeline([
    ('feature_selection', rfe),
    ('classifier', xgb_clf)
])

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(
    pipeline, X_train, y_train, 
    scoring='recall', cv=cv, n_jobs=-1
)
print('Recall (Validação Cruzada): %.3f (± %.3f)' % (np.mean(n_scores), np.std(n_scores)))

In [ ]:
pipeline.fit(X_train, y_train)

selected_features = X_train.columns[rfe.support_]
print("\nVariáveis selecionadas:")
print(selected_features.tolist())
print("Número de variáveis:", len(selected_features))

In [ ]:
X_train_sel = X_train[selected_features]
X_test_sel = X_test[selected_features]

## Treinamento e avaliação da regressão logística com variáveis selecionadas (RFE)

In [ ]:
results_lr_RFE, model_lr_RFE = train_logistic_regression(
    X_train_sel, X_test_sel , y_train, y_test,
    model_name="Regressão Logística (RFE)"
)

In [ ]:
results = pd.concat([results_lr, results_lr_RFE], ignore_index=True)
print(results.to_string(index=False))

## Cross Validation

In [ ]:
print("Validação Cruzada Regressão Logística:")
print(evaluate_model(
    LogisticRegression(class_weight='balanced'), 
    X_train_sel, 
    y_train
))

# Feature selection com SelectKBest

## Avaliação do desempenho do modelo para diferentes números de features selecionadas pelo SelectKBest

In [ ]:
scores = []
k_values = range(1, X_train.shape[1] + 1)

for k in k_values:
    pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('select', SelectKBest(score_func=f_classif, k=k)),
        ('clf', LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42))
    ])
    
    f1 = cross_val_score(pipe, X_train, y_train, cv=5, scoring='f1')
    scores.append(f1.mean())

best_k = k_values[np.argmax(scores)]
best_f1 = max(scores)
print(f"Melhor k: {best_k}, F1-score: {best_f1:.4f}")

In [ ]:
best_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('select', SelectKBest(score_func=f_classif, k=best_k)),
    ('clf', LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42))
])

best_pipe.fit(X_train, y_train)

selected_mask = best_pipe.named_steps['select'].get_support()
selected_features = X_train.columns[selected_mask]
print(f"Features selecionadas ({len(selected_features)}):", selected_features.tolist())

## Plotando a curva F1-score x número de features selecionadas

In [ ]:
best_k = k_values[np.argmax(scores)]
best_f1 = max(scores)

plt.plot(k_values, scores)
plt.axvline(x=best_k, color='r', linestyle=':', label=f'Melhor k={best_k} (F1={best_f1:.4f})')
plt.xlabel('Número de Features (k)')
plt.ylabel('F1-score')
plt.legend()
plt.grid()
plt.show()

## Seleção final de features usando SelectKBest com k=19

In [ ]:
X_train_kbest = best_pipe[:-1].transform(X_train)
X_test_kbest = best_pipe[:-1].transform(X_test)

# Treinamento e avaliação com SelectKBest

## **Regressão Logística**

In [ ]:
results_lr_KBest, model_lr_KBest = train_logistic_regression(
    X_train_kbest, X_test_kbest , y_train, y_test,
    model_name="Regressão Logística (SelectKBest)",
    scaling=False
)

## Comparando

In [ ]:
results = pd.concat([results_lr, results_lr_RFE, results_lr_KBest], ignore_index=True)
print(results.to_string(index=False))

## Cross Validation

In [ ]:
print("Validação Cruzada Regressão Logística (SelectKBest):")
print(evaluate_model(
    LogisticRegression(class_weight='balanced'), 
    X_train_kbest, 
    y_train,
    scaling=False
))

## **SVC**

In [ ]:
results_svc_kbest, model_svc_kbest = train_svc(
    X_train_kbest, X_test_kbest , y_train, y_test, 
    "SVC (SelectKBest)",
    scaling=False)

In [ ]:
results = pd.concat([results_svc, results_svc_kbest], ignore_index=True)
print(results.to_string(index=False))

## Cross Validation

In [ ]:
print("Relatório de classificação - SVC (SelectKBest):")
print(evaluate_model(model_svc_kbest, X_train_kbest, y_train, scaling=False))

## **KNN**

In [ ]:
results_knn_kbest, model_knn_kbest = train_knn(
    X_train_kbest, X_test_kbest , y_train, y_test, 
    "KNN (SelectKBest)",
    scaling=False
)

In [ ]:
results = pd.concat([results_knn_kbest, results_knn], ignore_index=True)
print(results.to_string(index=False))

## Cross Validation

In [ ]:
print("Relatório de classificação - KNN (SelectKBest):")
print("Validação Cruzada KNN:")
print(evaluate_model(
    KNeighborsClassifier(n_neighbors=3), 
    X_train_kbest, 
    y_train,
    scaling=False
))

## **Redução de dimensionalidade com PCA**
Dado o alto número de features correlacionadas, utilizei a estratégia de redução de dimensionalidade.

## Separação e padronização dos dados

In [ ]:
scaler = StandardScaler()
X_train_pca_scaled = scaler.fit_transform(X_train)
X_test_pca_scaled = scaler.transform(X_test)

## Análise da variância explicada

In [ ]:
pca = PCA()
pca.fit(X_train_pca_scaled)

cumulative_variance = np.cumsum(pca.explained_variance_ratio_)

plt.figure(figsize=(10, 6))
plt.plot(cumulative_variance, marker='o', label='Variância explicada acumulada')
plt.axhline(y=0.95, color='r', linestyle='--', label='Limite: 95% da variância')
plt.xlabel('Número de componentes')
plt.ylabel('Variância explicada acumulada')
plt.title('Variância explicada acumulada pelo PCA')
plt.legend(loc='lower right')
plt.grid(True)
plt.show()

## Aplicação do PCA com número otimizado de componentes

In [ ]:
pca = PCA(n_components=0.95)
pca.fit(X_train_pca_scaled)

In [ ]:
X_train_pca = pca.transform(X_train_pca_scaled)
X_test_pca = pca.transform(X_test_pca_scaled)

# Modelagem após PCA

## **Regressão Logística**

In [ ]:
results_lr_pca, model_lr_pca = train_logistic_regression(
    X_train_pca, X_test_pca , y_train, y_test, 
    "Regressão Logística (PCA)",
    scaling=False)

In [ ]:
results = pd.concat([results_lr, results_lr_RFE, results_lr_KBest, results_lr_pca], ignore_index=True)
print(results.to_string(index=False))

## Cross Validation

In [ ]:
print("Relatório de classificação - Regressão Logística com PCA:")
print(evaluate_model(model_lr_pca, X_train_pca, y_train, scaling=False))

## **SVC**

In [ ]:
results_svc_pca, model_svc_pca = train_svc(
    X_train_pca, X_test_pca , y_train, y_test, 
    "SVC (PCA)",
    scaling=False    
)

In [ ]:
results = pd.concat([results_svc_pca, results_svc_kbest, results_svc], ignore_index=True)
print(results.to_string(index=False))

## Cross Validation

In [ ]:
print("Relatório de classificação - SVC com PCA:")
print(evaluate_model(model_svc_pca, X_train_pca, y_train, scaling=False))

## **KNN**

In [ ]:
results_knn_pca, model_knn_pca = train_knn(
    X_train_pca, X_test_pca , y_train, y_test, 
    "KNN (PCA)", 
    scaling=False
)

In [ ]:
results = pd.concat([results_knn_kbest, results_knn, results_knn_pca], ignore_index=True)
print(results.to_string(index=False))

## Cross Validation

In [ ]:
print("Relatório de classificação - KNN com PCA:")
print(evaluate_model(model_knn_pca, X_train_pca, y_train, scaling=False))

## Comparação de modelos

In [ ]:
results = pd.concat([
    results_lr, results_rf, results_xgb, 
    results_xgb_opt, results_svc, results_knn,
    results_lr_RFE, results_lr_KBest,
    results_svc_kbest, results_knn_kbest,
    results_lr_pca, results_svc_pca, results_knn_pca
], ignore_index=True)

results.insert(0, 'Técnica', ['FS Corr']*6 + ['RFE']*1 + ['SelectKBest']*3 + ['PCA']*3)
results.index = range(1, len(results)+1)

final_results = (results.style
             .highlight_max(subset=['Accuracy', 'Recall', 'Precision', 'F1 Score'], color='#d4edda')
             .highlight_max(subset=['AUC'], color='#cce5ff')
             .format({
                 'Accuracy': '{:.4f}',
                 'Recall': '{:.4f}',
                 'Precision': '{:.4f}',
                 'F1 Score': '{:.4f}',
                 'AUC': '{:.4f}'
             })
             .set_caption('Comparação de Modelos - Métricas de Performance'))

final_results

## Gráfico de comparação de desempenho dos modelos

In [ ]:
results_long = results.melt(id_vars='Modelo', 
                            value_vars=['Accuracy', 'Recall', 'Precision', 'F1 Score', 'AUC'],
                            var_name='Métrica', 
                            value_name='Valor')

plt.figure(figsize=(12, 8))
sns.barplot(data=results_long, y='Modelo', x='Valor', hue='Métrica')
plt.title('Comparação de Desempenho dos Modelos', fontsize=16)
plt.xlim(0.9, 1.0)
plt.xlabel('Pontuação', fontsize=12)
plt.ylabel('Modelos', fontsize=12)
plt.xticks(fontsize=11)
plt.yticks(fontsize=11)
plt.legend(title='Métrica', bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()

## Conclusão final sobre a seleção do modelo
Diante do objetivo central deste projeto — **minimizar falsos negativos** na detecção de câncer de mama —, priorizei a métrica de Recall como fator decisivo. Uma classificação incorreta de um caso maligno (falso negativo) pode ter consequências críticas. Após avaliação rigorosa de múltiplas técnicas e algoritmos, o modelo **XGBoost (Padrão ou com Hiperparâmetros Otimizados)** emergiu como a melhor solução, oferecendo o melhor equilíbrio entre precisão diagnóstica e segurança clínica.


### Destaques do modelo selecionado: XGBoost (hiperparâmetros otimizados)
- **Recall**: 0.9767 – detectou praticamente todos os casos positivos
- **Precisão**: 1.0 – nenhuma previsão positiva foi incorreta
- **F1 Score**: 0.9882 – melhor equilíbrio entre sensibilidade e precisão
- **AUC**: 0.9964 – excelente capacidade de separação entre classes

### Vantagens-chave
- Reduz o risco de diagnósticos fatais (apenas 1 falso negativo em 114 casos)
- Nenhum falso positivo (100% de precisão)
- Excelente equilíbrio entre métricas
- Forte capacidade discriminativa (AUC próximo de 1)
- Consistência entre validação cruzada e conjunto de teste
- Amplo uso e desempenho confiável em tarefas reais de classificação

## Preparação para deploy
Para tornar o projeto reprodutível e aplicável, o modelo final foi encapsulado em um Pipeline com pré-processamento padronizado, treinado novamente no conjunto de treino e salvo em models/xgboost_breast_cancer_fs_optimized.pkl. Esse arquivo é carregado no aplicativo interativo (app.py), permitindo prever novos casos de forma simples e consistente.

In [ ]:
metrics_df, trained_xgb, search = train_xgboost(
    X_train_fs, X_test_fs, y_train, y_test,
    optimize_hyperparams=True,
    n_iter=100,
    cv=10
)

final_pipeline = Pipeline(steps=[
    ('classifier', trained_xgb)
])

final_pipeline.fit(X_train_fs, y_train)

joblib.dump(final_pipeline, "../models/xgboost_breast_cancer_fs_optimized.pkl")
print("Salvo em models/xgboost_breast_cancer_fs_optimized.pkl")

## Créditos e contato

**Desenvolvido por:**  
Bruno Casini

**GitHub:**  
[<img src="https://img.icons8.com/ios-filled/20/000000/github.png"/> GitHub](https://github.com/kzini)  
`https://github.com/kzini`

**LinkedIn:**  
<img src="https://img.icons8.com/ios-filled/20/000000/linkedin.png"/> Em construção 